In [ ]:
pip install -q flair

     |████████████████████████████████| 450kB 5.6MB/s 
     |████████████████████████████████| 798kB 15.6MB/s 
     |████████████████████████████████| 983kB 53.6MB/s 
     |████████████████████████████████| 1.1MB 52.1MB/s 
     |████████████████████████████████| 19.7MB 1.4MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 1.3MB 53.3MB/s 
     |████████████████████████████████| 2.9MB 55.0MB/s 
     |████████████████████████████████| 890kB 56.2MB/s 


In [ ]:
import urllib.request
from pathlib import Path

def download_file(url, output_file):
  Path(output_file).parent.mkdir(parents=True, exist_ok=True)
  urllib.request.urlretrieve (url, output_file)

download_file('https://raw.githubusercontent.com/ZihanWangKi/CrossWeigh/master/data/conllpp_train.txt', '/content/data/conllpp_train.txt')
download_file('https://raw.githubusercontent.com/ZihanWangKi/CrossWeigh/master/data/conllpp_dev.txt', '/content/data/conllpp_dev.txt')
download_file('https://raw.githubusercontent.com/ZihanWangKi/CrossWeigh/master/data/conllpp_test.txt', '/content/data/conllpp_test.txt')

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
columns = {0: 'text', 3: 'ner'}
corpus: Corpus = ColumnCorpus('/content/data/', columns,
                              train_file='conllpp_train.txt',
                              test_file='conllpp_test.txt',
                              dev_file='conllpp_dev.txt')

2021-03-02 11:31:58,717 Reading data from /content/data
2021-03-02 11:31:58,718 Train: /content/data/conllpp_train.txt
2021-03-02 11:31:58,723 Dev: /content/data/conllpp_dev.txt
2021-03-02 11:31:58,724 Test: /content/data/conllpp_test.txt


In [ ]:
import pandas as pd
data = [[len(corpus.train), len(corpus.test), len(corpus.dev)]]
# Prints out the dataset sizes of train test and development in a table.
pd.DataFrame(data, columns=["Train", "Test", "Development"])

,Train,Test,Development
0,14987,3684,3466


In [ ]:
import flair
from typing import List
from flair.trainers import ModelTrainer
from flair.models import SequenceTagger
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings

tag_type = 'ner'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

# For faster training and smaller models, we can comment out the flair embeddings.
# This will significantly affect the performance though.
embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train('/content/model/conllpp',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=50,
              embeddings_storage_mode='gpu')

2021-03-02 11:32:12,827 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmp49gcvefo


100%|██████████| 160000128/160000128 [00:17<00:00, 9363539.54B/s] 

2021-03-02 11:32:30,734 copying /tmp/tmp49gcvefo to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2021-03-02 11:32:30,952 removing temp file /tmp/tmp49gcvefo
2021-03-02 11:32:32,065 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpof2d8s3n


100%|██████████| 21494764/21494764 [00:03<00:00, 5972507.93B/s]

...

2021-03-02 11:33:05,126 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmpxdyrd7zw


100%|██████████| 73034575/73034575 [00:08<00:00, 8645690.92B/s]

2021-03-02 11:33:14,402 copying /tmp/tmpxdyrd7zw to cache at /root/.flair/embeddings/news-backward-0.4.1.pt
2021-03-02 11:33:14,467 removing temp file /tmp/tmpxdyrd7zw


2021-03-02 11:33:15,839 ----------------------------------------------------------------------------------------------------
2021-03-02 11:33:15,840 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4196, out_features=4196, bias=True)
  (rnn): LSTM(4196, 256, batch_first=True, b

{'dev_loss_history': [0.9109666347503662,
  0.6897583603858948,
  0.6010030508041382,
  0.5215756297111511,
  0.4869338572025299,
  0.4407424032688141,
  0.45779162645339966,
  0.41924217343330383,
  0.4233095049858093,
  0.4095763862133026,
  0.4145328104496002,
  0.40496698021888733,
  0.37435823678970337,
  0.4006655514240265,
  0.3789050877094269,
  0.3682231903076172,
  0.3839879631996155,
  0.3926396667957306,
  0.38985511660575867,
  0.3862716555595398,
  0.38662225008010864,
  0.379253625869751,
  0.3878098726272583,
  0.4129790961742401,
  0.3732573986053467,
  0.4081420600414276,
  0.39106860756874084,
  0.3763274848461151,
  0.38706260919570923,
  0.3858925402164459,
  0.39461028575897217,
  0.3834135830402374,
  0.38297325372695923,
  0.4007221758365631,
  0.395418643951416,
  0.3955330550670624,
  0.3976260721683502,
  0.38740694522857666,
  0.39020565152168274,
  0.3880842924118042,
  0.3969576954841614,
  0.40057605504989624,
  0.40254509449005127,
  0.406009316444397,
 

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

input_sentence = 'My name is Eugene, I currently live in Singapore, I work for DSO.'
tagger: SequenceTagger = SequenceTagger.load("/content/model/conllpp/final-model.pt")
sentence: Sentence = Sentence(input_sentence)
tagger.predict(sentence)
print(sentence.to_tagged_string())

2021-03-02 12:37:05,917 loading file /content/model/conllpp/final-model.pt
My name is Eugene <B-PER> , I currently live in Singapore <B-LOC> , I work for DSO <B-ORG> .


REFERANS:
[1] https://news.machinelearning.sg/posts/train_a_named_entity_recognition_model_using_flair/
